# Parallel Wiener-Hammerstein system

See http://arxiv.org/pdf/1708.06543 for more information on the dataset.

Note that running this file multiple times, or on different machines, with the same parameters may yield different results due to the non-deterministic nature of GPU parallel execution.

In [1]:
import matplotlib.pyplot as plt
import nonlinear_benchmarks as nlb
import numpy as np
import optimistix as optx
from nonlinear_benchmarks.error_metrics import RMSE

import freq_statespace as fss


# Load data
ParWH_full_train, ParWH_full_test = nlb.ParWH() 

# Initialize variables
N = 16384  # number of samples per period
R = 5  # number of random phase multisine realizations
P = 2  # number of periods
amplitude_level = 4  # must be one of {0, 1, 2, 3, 4}

nu, ny = 1, 1  # SISO system

fs = 78e3  # [Hz]
f_idx = np.arange(1, 4096)  # frequency lines of interest (excludes DC)

# Load data
ParWH_full_train, ParWH_full_test = nlb.ParWH() 
ParWH_train = [
    data for data in ParWH_full_train
    for phase in range(R)
    if data.name == f'Est-phase-{phase}-amp-{amplitude_level}'
]
ParWH_test = [
    data for data in ParWH_full_test
    if data.name == f'Val-amp-{amplitude_level}'
][0]

# Preprocess data
u_train = np.array([data.u for data in ParWH_train]).reshape(R, nu, N, P)
y_train = np.array([data.y for data in ParWH_train]).reshape(R, ny, N, P)
u_train = np.transpose(u_train, (2, 1, 0, 3))
y_train = np.transpose(y_train, (2, 1, 0, 3))

u_test = np.transpose(ParWH_test.u.reshape(1, nu, N, 2), (2, 1, 0, 3))
y_test = np.transpose(ParWH_test.y.reshape(1, ny, N, 2), (2, 1, 0, 3))

# Create input-output training data object
data = fss.create_data_object(u_train, y_train, f_idx, fs)

Step 1: Best Linear Approximation

In [2]:
# (i) Parametrize using frequency-domain subspace identification method
nx = 12  # number of states
q = 14 if amplitude_level == 0 else 13  # larger q at lowest level for stability 
bla_fsid = fss.lin.subspace_id(data, nx, q)

# (ii) Frequency-domain iterative optimization starting from FSID BLA
max_iter = 10 if amplitude_level == 0 else 100
solver = optx.LevenbergMarquardt(rtol=1e-3, atol=1e-6)
bla_opti = fss.lin.optimize(bla_fsid, data, solver=solver, max_iter=max_iter)

=============== Frequency-domain subspace identification ===============
BLA simulation error: 13.31%.

========================== BLA optimization  ===========================
Starting iterative optimization...
    Iter 1 | loss = 4.3606e-01
    Iter 2 | loss = 3.4948e-01
    Iter 3 | loss = 3.4948e-01
    Iter 4 | loss = 3.3386e-01
    Iter 5 | loss = 3.2910e-01
    Iter 6 | loss = 3.1108e-01
    Iter 7 | loss = 3.0547e-01
    Iter 8 | loss = 3.0059e-01
    Iter 9 | loss = 2.9576e-01
    Iter 10 | loss = 2.9069e-01
    Iter 11 | loss = 2.8645e-01
    Iter 12 | loss = 2.8389e-01
    Iter 13 | loss = 2.8270e-01
    Iter 14 | loss = 2.8211e-01
    Iter 15 | loss = 2.8172e-01
    Iter 16 | loss = 2.8143e-01
    Iter 17 | loss = 2.8117e-01
    Iter 18 | loss = 2.8094e-01
    Iter 19 | loss = 2.8071e-01
    Iter 20 | loss = 2.8047e-01
    Iter 21 | loss = 2.8022e-01
    Iter 22 | loss = 2.7992e-01
    Iter 23 | loss = 2.7957e-01
    Iter 24 | loss = 2.7909e-01
    Iter 25 | loss = 2.7840e-

Step 2: inference and learning

In [ ]:
# Define the nonlinear basis function
degree = 7
nw = 2
nz = 2
phi = fss.f_static.basis.Polynomial(nz, degree)

# As for the solver, we typically prefer a dedicated least-squares solver,
# such as the Levenberg–Marquardt (LM) algorithm. However, these solvers tend to have
# a high memory footprint, which can quickly become prohibitive for large-scale problems
# (like the one at hand). Your machine may not have enough memory to run it.
#
# This is especially true for our inference and learning approach, which involves:
# - fixed-point iterations within the loss function,
# - large feature matrices from nonlinear basis expansions,
# - multiple FFT operations and reshaping steps,
# all of which contribute to a large autodiff trace and memory pressure under LM.
#
# Fortunately, we can also opt for general-purpose minimizers that are more
# memory-efficient, such as BFGS or any first-order optimizer from the Optax library!
# These only require the gradient of a scalar loss and avoid tracing through the full
# residual or Jacobian.
#
# Example solver choices:
# solver = optx.LevenbergMarquardt(rtol=1e-3, atol=1e-6)
# solver = optx.OptaxMinimiser(optax.adam(learning_rate=1e-3), rtol=1e-3, atol=1e-6)
# solver = optx.BFGS(rtol=1e-3, atol=1e-6)

# Solve the problem
solver = optx.BFGS(rtol=1e-3, atol=1e-6)
lambda_w = 1e2 if amplitude_level == 0 else 1  # more regularization at lowest level due to poor SNR
nllfr_init = fss.nonlin.inference_and_learning(
    bla_opti, data, phi=phi, nw=nw, lambda_w=lambda_w, fixed_point_iters=15, solver=solver
)

==================== NL-LFR inference and learning  ====================
Starting iterative optimization...
    BLA loss: 4.0268e+04


2025-11-18 17:24:42.449792: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3023] Can't reduce memory use below 2.59GiB (2776942192 bytes) by rematerialization; only reduced to 2.66GiB (2860186931 bytes), down from 2.66GiB (2860516011 bytes) originally
2025-11-18 17:24:54.812137: W external/xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.66GiB (rounded to 2860845312)requested by op 
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-11-18 17:24:54.812312: W external/xla/xla/tsl/framework/bfc_allocator.cc:512] *_*_________________________________________________________________________________________________
E1118 17:24:54.812339  113650 pjrt_stream_executor_client.cc:2916] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying

XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 2860845160 bytes.

Step 3: full nonlinear optimization

In [ ]:
# Regarding the solver, the same story as above applies. But this optimization step is a
# bit less  memory-hungry than inference and learning, so we can afford to use a more 
# powerful solver.
#
# Example solver choices:
# solver = optx.LevenbergMarquardt(rtol=1e-3, atol=1e-6)
# solver = optx.OptaxMinimiser(optax.adam(learning_rate=1e-3), rtol=1e-3, atol=1e-6)
# solver = optx.BFGS(rtol=1e-3, atol=1e-6)

# Solve the problem
solver = optx.LevenbergMarquardt(rtol=1e-3, atol=1e-6)
nllfr_opti = fss.nonlin.optimize(nllfr_init, data, solver=solver, max_iter=250)

Evaluate the model on test data

In [ ]:
# Prepare test data
u_test = ParWH_test.u.reshape(N, 2).T.flatten()  # 2 periods in a single vector
y_test = ParWH_test.y.reshape(N, 2).T.flatten()  # 2 periods in a single vector

# Simulate BLA and NL-LFR models
y_test_bla = bla_opti.simulate(u_test)[0]
y_test_nllfr = nllfr_opti.simulate(u_test)[0]
        
# Check performance (we only check the second period)
y_test_plot = y_test[N:]
e_test_bla = y_test_plot - y_test_bla[N:]
e_test_nllfr = y_test_plot - y_test_nllfr[N:]

mse_bla = np.mean((e_test_bla)**2)
mse_nllfr_lfr = np.mean((e_test_nllfr)**2)
norm = np.mean(y_test_plot**2)

print(f'NRMSE of BLA model on test data: {100 * np.sqrt(mse_bla / norm):.2f}%')
print(f'NRMSE of NL-LFR model on test data: {100 * np.sqrt(mse_nllfr_lfr / norm):.2f}%')

# Convert to frequency domain
Y_test = np.fft.rfft(y_test_plot, norm='forward')
E_test_bla = np.fft.rfft(e_test_bla, norm='forward')
E_test_nonlin_lfr = np.fft.rfft(e_test_nllfr, norm='forward')

# Compute noise level
Y_noise_std = np.sqrt(data.freq.Y_var_noise)
Y_noise_std = 1 / N * Y_noise_std * data.norm.y_std  # denormalize

t = data.time.t
f = data.freq.f[f_idx]

fig, axs = plt.subplots(1, 2, figsize=(9, 4))
axs[0].plot(t, y_test_plot, label='system output')
axs[0].plot(t, e_test_bla, label='BLA error')
axs[0].plot(t, e_test_nllfr, label='NL-LFR error')
axs[0].set_title('Multisine - Time Domain')
axs[0].set_xlabel('time [s]')
axs[0].set_ylabel('output [V]')
axs[0].legend()

axs[1].plot(f, 20*np.log10(np.abs(Y_test[f_idx])), label='system output')
axs[1].plot(f, 20*np.log10(np.abs(E_test_bla[f_idx])), label='BLA error')
axs[1].plot(f, 20*np.log10(np.abs(E_test_nonlin_lfr[f_idx])), label='NL-LFR error')
axs[1].plot(f, 20*np.log10(Y_noise_std[f_idx]), 'k', label='noise level')
axs[1].set_title('Multisine - Frequency Domain')
axs[1].set_xlabel('frequency [Hz]')
axs[1].set_ylabel('magnitude [dB]')
axs[1].legend()
plt.tight_layout()
plt.show()


In [ ]:
# Official reporting for nonlinear benchmark submission
state_init_window_length = 50  # found in recent nonlinear_benchmark commit `c74fb1b`
rmse = RMSE(y_test[state_init_window_length:], y_test_nllfr[state_init_window_length:])
print(f'Amplitude level {amplitude_level}, test RMSE: {1000 * rmse:.4f} mV')